# A ReAct agent with Command R+, achieving the same goals as Adaptive RAG | Add the custom tools

Adaptive RAG is a strategy for RAG that unites (1) [query analysis](https://blog.langchain.dev/query-construction/) with (2) [active / self-corrective RAG](https://blog.langchain.dev/agentic-rag-with-langgraph/).

In the paper, they report query analysis to route across:
- No Retrieval (LLM answers)
- Single-shot RAG
- Iterative RAG


We'll use Command R+, a recent release from Cohere that:
- Has strong accuracy on RAG, Tool Use and Agents
- Has 128k context


# Environment

In [4]:
! pip install --quiet langchain langchain_cohere tiktoken faiss-cpu beautifulsoup4 langchain_experimental matplotlib tabulate

In [3]:
### LLMs
import os

os.environ["COHERE_API_KEY"] = "<key>"

# Create tools
The ReAct agent will be equipped with these tools. The model can pick between
- web search
- RAG: retrieval from a vector store
- custome tool (call an external API)
- directly answering

The model can use any of these tools, in any sequence of steps, and self-reflect.

### Web search tool

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field

os.environ["TAVILY_API_KEY"] = "<key>"

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")


internet_search.args_schema = TavilySearchInput

### Python interpreter tool

In [7]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

# from langchain_core.pydantic_v1 import BaseModel, Field
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
repl_tool.args_schema = ToolInput

### RAG tool

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

# Set embeddings
embd = CohereEmbeddings()

# Docs to index
urls = [
    "https://www.mayerbrown.com/en/insights/publications/2023/03/new-data-standards-pending-for-federally-regulated-financial-entities",
    "https://plaid.com/resources/open-finance/what-is-fdx/",
    "https://www.egnyte.com/guides/financial-services/financial-data-compliance",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=embd,
)

vectorstore_retriever = vectorstore.as_retriever()

In [9]:
from langchain.tools.retriever import create_retriever_tool

vectorstore_search = create_retriever_tool(
    retriever=vectorstore_retriever,
    name="vectorstore_search",
    description="Retrieve relevant info from a vectorstore that contains documents related to agents, prompt engineering, and adversarial attacks.",
)

### Custom Tool For Market Capitalization



Define the function

In [11]:
import requests

# Function to get the market capitalization of a ticker symbol
def get_market_cap(ticker):
    api_key = "<key>"
    url = f"https://financialmodelingprep.com/api/v3/market-capitalization/{ticker}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data and isinstance(data, list) and len(data) > 0:
            market_cap = data[0].get('marketCap', 'No market cap data available')
            return [{'id': 0, 'text': f'Market cap for {ticker}: ${market_cap}'}]
        else:
            return "No data available for the specified ticker."
    else:
        return "Failed to retrieve data from the API."

### Define Custom Tool

In [12]:
from langchain.tools import tool

@tool
def market_cap(ticker: str) -> list:
    """This tool is only used to find market capitalization. Do not use it for anything else. Find the ticker when asked about capitalization. The ticker symbol of the company (e.g., AAPL which is the ticker for Apple Inc, MSFT for Microsoft )"""
    return get_market_cap(ticker)
    

# Create the ReAct Agent
The model can smartly pick the right tool(s) for the user query, call them in any sequence of steps, analyze the results and self-reflect.

In [13]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [14]:
# LLM
from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(model="command-r-plus", temperature=0.3)

# Preamble
preamble = """
Use all tools that are available to answear the question. 
If the query covers the topics of Federal Financial Institutions, use the vectorstore search first.
You are equipped with an internet search tool and a special vectorstore of information about Data Compliance for Financial Services.

"""

# Prompt
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[vectorstore_search, internet_search, repl_tool, market_cap],
    prompt=prompt,
)

    # tools=[vectorstore_search, internet_search, repl_tool],

In [15]:
agent_executor = AgentExecutor(
    agent=agent, tools=[vectorstore_search, internet_search, repl_tool, market_cap], verbose=True
)

# Testing the ReAct agent

**Let's ask a question that requires web search.**

In [16]:
result = agent_executor.invoke(
    {
        "input": "What are the primary functions of the Federal Financial Institutions Examination Council (FFIEC)?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for the primary functions of the Federal Financial Institutions Examination Council (FFIEC) in the vector store.
{'tool_name': 'vectorstore_search', 'parameters': {'query': 'primary functions of the Federal Financial Institutions Examination Council (FFIEC)?'}}
cybersecurity by implementing best practices and employing proven cybersecurity frameworks. The importance of financial data compliance continues to grow as regulations continue to expand globally to ensure that customers’ personal information and finances are protected from theft and fraud.Advantages of Financial Data Compliance and Cost of Non-ComplianceBenefits of financial data complianceNot following financial data complianceView of the most critical data and systemsPlan for cybersecurity tools and practicesProtection of valuable informationRapid response to cybersecurity incidentsOperational disruptionsReputational damageCivil and criminal lawsuitsFines and losses due 

In [47]:
result["output"]

'The Federal Financial Institutions Examination Council (FFIEC) is a five-member U.S. government interagency body. It consists of five banking regulators and is responsible for uniformity in the guidelines and supervision of financial institutions, including financial data compliance. It includes financial data management mandates, such as directing how organisations should govern the secure storage of all types of sensitive information, whether on computer systems, physical media, or hard-copy documents.'

**Let's ask a question that requires RAG retrieval from the vector db.**

In [57]:
agent_executor.invoke(
    {
        "input": "How does the current interest rate environment affect the bond market?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for 'How does the current interest rate environment affect the bond market?' and write an answer based on the results.
{'tool_name': 'internet_search', 'parameters': {'query': 'How does the current interest rate environment affect the bond market?'}}
[{'url': 'https://investor.vanguard.com/investor-resources-education/news/the-dynamics-of-bond-duration-and-rising-rates', 'content': "When the investment horizon is longer than the bond's duration, however, higher yields on reinvested cash flow outweigh the market price decline. Over a period of 15, 20, or 25 years, interest rate rises of 100 and 200 basis points result in an improvement in total returns. (The inverse is true for total returns when interest rates decline."}, {'url': 'https://www.williamblair.com/Insights/How-Are-Rising-Interest-Rates-Affecting-the-Bond-Market', 'content': "Since the Fed began raising its target interest rate last year, rates have jumped to a 5%-5.25% 

{'input': 'How does the current interest rate environment affect the bond market?',
 'preamble': '\nUse all tools that are available to answear the question.\nIf the query covers the topics of Federal Financial Institutions, use the vectorstore search first.\nYou are equipped with an internet search tool and a special vectorstore of information about Data Compliance for Financial Services.\n\n',
 'output': 'The current interest rate environment has a significant impact on the bond market. When the Fed raises interest rates, it affects bond prices. For example, when the Fed raised its target interest rate last year, rates jumped to a 5%-5.25% range from near zero in March 2022, the fastest pace of increase in four decades. This rate hike cycle has impacted how investors are analysing risk/reward trade-offs.\n\nThe federal-funds rate, the interest rate at which banks lend money to each other overnight, is now targeted between 1.75% and 2.00%. \n\nThe yield curve, which represents differe

In [66]:
agent_executor.invoke(
    {
        "input": "What is Apple capitalization compared to Microsoft? Calculate the difference",
         "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will use the market_cap tool to find the market capitalisation of Apple and Microsoft. Then, I will calculate the difference between the two.
{'tool_name': 'market_cap', 'parameters': {'ticker': 'AAPL'}}
[{'id': 0, 'text': 'Market cap for AAPL: $2609989938000'}]
{'tool_name': 'market_cap', 'parameters': {'ticker': 'MSFT'}}
[{'id': 0, 'text': 'Market cap for MSFT: $3039495786400'}]
Apple's market cap is $2609989938000 and Microsoft's is $3039495786400. Now I will calculate the difference between the two.
{'tool_name': 'python_interpreter', 'parameters': {'code': 'apple_cap = 2609989938000\nmsft_cap = 3039495786400\n\ndifference = msft_cap - apple_cap\n\nprint(f"The difference between Apple and Microsoft\'s market cap is {difference}")'}}
The difference between Apple and Microsoft's market cap is 429505848400
Relevant Documents: 0,1,2
Cited Documents: 0,1,2
Answer: Apple's market cap is $2,609,989,938,000 and Microsoft's is $3,039,495,786,400. T

{'input': 'What is apple capitalization compared to Microsoft? Calculate the difference',
 'preamble': '\nUse all tools that are available to answear the question.\nIf the query covers the topics of Federal Financial Institutions, use the vectorstore search first.\nYou are equipped with an internet search tool and a special vectorstore of information about Data Compliance for Financial Services.\n\n',
 'output': "Apple's market cap is $2,609,989,938,000 and Microsoft's is $3,039,495,786,400. The difference between the two is $429,505,848,400.",
 'citations': [CohereCitation(start=22, end=40, text='$2,609,989,938,000', documents=[{'id': 'doc_0', 'text': 'Market cap for AAPL: $2609989938000'}], document_ids={'doc_0'}),
  CohereCitation(start=60, end=78, text='$3,039,495,786,400', documents=[{'id': 'doc_1', 'text': 'Market cap for MSFT: $3039495786400'}], document_ids={'doc_1'})]}

In [14]:
agent_executor.invoke(
    {
        "input": "What is the share value of Microsoft stock today?",
         "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for the share value of Microsoft stock today.
{'tool_name': 'internet_search', 'parameters': {'query': 'share value of Microsoft stock today'}}
[{'url': 'https://www.cnn.com/markets/stocks/MSFT', 'content': 'The Productivity and Business Processes segment consists of Office Commercial (Office 365 subscriptions, the Office 365 portion of Microsoft 365 Commercial subscriptions, and Office licensed on-premises), Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, Office Consumer, including Microsoft 365 Consumer subscriptions, Office licensed on-premises, and other Office services, LinkedIn, including Talent Solutions, Marketing Solutions, Premium Subscriptions, Sales Solutions, and Learning Solutions, Dynamics business solutions, including Dynamics 365, comprising a set of intelligent, cloud-based applications across ERP, CRM, Customer Insights, Power Apps, and Power Automate, and on-pre

{'input': 'What is the share value of Microsoft stock today?',
 'preamble': '\nUse all tools that are available to answear the question. \nIf the query covers the topics of Federal Financial Institutions, use the vectorstore search first.\nYou are equipped with an internet search tool and a special vectorstore of information about Data Compliance for Financial Services.\n\n',
 'output': 'I was unable to find the share value of Microsoft stock today. However, I did find that the share price of Microsoft stock is 169.18 or 170.48.',
 'citations': [CohereCitation(start=126, end=132, text='169.18', documents=[{'url': 'https://finance.yahoo.com/quote/MSFT;/', 'content': '169.18. -0.76%. Find the latest Microsoft Corporation (MSFT;) stock quote, history, news and other vital information to help you with your stock trading and investing.', 'id': 'doc_2'}, {'url': 'https://finance.yahoo.com/quote/MSFT;/', 'content': '169.18. -0.76%. Find the latest Microsoft Corporation (MSFT;) stock quote, hi